In [1]:
from functools import wraps, reduce

In [2]:
def multireturn(function):

    function_returns = {}
    function_returns['generator'] = lambda gen: gen
    function_returns['list'] = lambda gen: list(gen)
    function_returns['value'] = lambda gen: reduce(lambda a, b: b, gen)

    @wraps(function)
    def wrapper(*args, **kwargs):

        mod_args = list(args)
        returns = 'value'
        if 'returns' in kwargs.keys():
            returns = kwargs['returns']
            del kwargs['returns']
            mod_args[0] = (mod_args[0] - 1) \
                if returns in ['generator', 'list'] else mod_args[0]

        try:
            return function_returns[returns](function(*mod_args, **kwargs))
        except KeyError:
            raise ValueError('"returns" must be: "value", '
                             '"list" or "generator"')

    return wrapper

In [3]:
@multireturn
def factorial(n):
    a = 1
    if n >= 0:
        yield a
    for i in range(1, n + 1):
        a *= i
        yield a

@multireturn
def fibonacci(n):
    a, b = (0, 1)
    if n >= 0:
        yield a
    for i in range(n):
        a, b = b, a + b
        yield a

@multireturn
def get_prime(n, found=[2, 3, 5, 7, 11, 13, 17]):
    already = len(found)
    for i in range(n + 1 if n < already else already):
        yield found[i]
    counter = found[-1] if already > 1 else 1
    for i in range(already, n + 1):
        while True:
            counter += 2
            prime = True
            for number in found:
                if counter % number == 0:
                    prime = False
                    break
            if prime:
                found.append(counter)
                yield found[-1]
                break